# Vanilla GAN

- Referecnce : https://github.com/eriklindernoren/PyTorch-GAN

In [ ]:
import argparse
import os
import numpy as np
import math

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Define Generator & Discriminator
- layers would be mlp
- Activations except last one are nn.LeakyReLU(0.2)
- BachNorm parameter = 0.8
- Dropout p = 0.5

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        #### Implement Here ####
        

    def make_block(self, in_feat, out_feat):
        #### Implement Here ####
        

        return 

    def forward(self, z):
        #### Implement Here ####
        # Hint : Embedding vector should be resized into image shape after go through network
        
        
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        #### Implement Here ####

        
    def forward(self, img):
        #### Implement Here ####
        # Hint : Image should be vectorized before go through network
        

        return x

In [ ]:
# Initialize generator and discriminator
G = Generator()
D = Discriminator()

In [ ]:
from torchsummary import summary
summary(G, device = 'cpu', batch_size = -1, input_size = (100,))
summary(D, device = 'cpu', batch_size = -1, input_size = (1, 28, 28))

## Define loss & Optimizer & model

In [ ]:
# Loss function
criterion = torch.nn.BCELoss()

# Optimizer for each network
optimizer_G = torch.optim.Adam(G.parameters(), lr=0.0001, betas = (0.5, 0.9999))
optimizer_D = torch.optim.Adam(D.parameters(), lr=0.0001, betas = (0.5, 0.9999))

## Use MNIST Dataset & DataLoader

In [ ]:
train_bs = 64

dataset = datasets.MNIST("./mnist", train=True, download=True, 
                         transform=transforms.Compose([transforms.Resize(28), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=train_bs, shuffle=True, drop_last=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

batch = next(iter(dataloader))
len(batch)
print(batch[0].shape, batch[1])

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.figure(figsize=(20,10))
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()
    
imshow(torchvision.utils.make_grid(batch[0]))

## Start Training

In [ ]:
G, D = G.to(device), D.to(device)
os.makedirs("./mlpgan", exist_ok=True)
epochs = 200

In [ ]:
for epoch in range(epochs):
    for i, (real_imgs, _) in enumerate(dataloader):

        # -----------------
        #  Train Generator
        # -----------------
        real_imgs = real_imgs.to(device)

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.Tensor(np.random.normal(0, 1, (real_imgs.shape[0], 100))).to(device)

        # Generate a batch of images
        gen_imgs = G(z)
        
        # Loss measures generator's ability to fool the discriminator
        #### implement Loss ####
        

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        #### implement Loss ####

        d_loss.backward()
        optimizer_D.step()

        batches_done = epoch * len(dataloader) + i
        
        if batches_done % 100 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch+1,epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )

        batches_done = epoch * len(dataloader) + i
        if batches_done % 2000 == 0:
            save_image(gen_imgs.data, "mlpgan/%06d.png" % batches_done, nrow=8, normalize=True)
            imshow(torchvision.utils.make_grid(gen_imgs.cpu()))